## 13 물체 감지와 추적

- Opencv 설치
- 프레임 차이 계산법
- 색 공간을 이용한 물체 추적 기법
- 배경 분리 기법을 이용한 물체 추적 기법
- 캠시프트 알고리즘으로 인터렉티브 방식 물체 추적기를 만드는 방법
- 광학흐름 기반 추적 기법

### 0. opencv 설치 가이드
https://www.learnopencv.com/install-opencv3-on-macos/

### 테스트

In [1]:
import sys

In [2]:
sys.path.append('/usr/local/lib/python3.6/site-packages')

In [3]:
import cv2
print(cv2.__version__)

3.4.0


### 1. 프레임 차이 대조법
(https://www.youtube.com/watch?v=DrCsvqE18u0)

- 실시간 비디오 스트림에서 캡처한 연속된 프레임들의 차이점을 분석 >> 움직이는 부분을 탐지하는 기법
- 노이즈에 굉장히 민감 >> 물체를 정확히 추적하기 힘듦

#### 주요 함수
- frame_diff( prev_frame, cur_frame, next_frame )
    - <b>cv2.absdiff</b> : Calculates the per-element absolute difference between two arrays or between an array and a scalar.
    - <b>cv2.bitwise</b> : Calculates the per-element bit-wise conjunction of two arrays or an array and a scalar.

- get_frame( cap, scaling_factor )
    - <b>resize</b> : Resizes an image.
    - <b>cvtColor</b>


출처 : https://docs.opencv.org/2.4/modules/core/doc/operations_on_arrays.html

In [24]:
import cv2

# Compute the frame differences
def frame_diff(prev_frame, cur_frame, next_frame):
    # 현재 프레임과 다음 프레임의 엘리먼트별 절대값 차이 구하기
    diff_frames_1 = cv2.absdiff(next_frame, cur_frame)

    # 현재 프레임과 이전 프레임의 엘리먼트별 절대값 차이 구하기
    diff_frames_2 = cv2.absdiff(cur_frame, prev_frame)

    return cv2.bitwise_and(diff_frames_1, diff_frames_2) ## 두 차이점을 비트단위로 AND 연산을 수행 후 결과 반환



# Define a function to get the current frame from the webcam
def get_frame(cap, scaling_factor):
    # 현재 비디오에서 프레임 읽기
    _, frame = cap.read()

    # 이미지 크기 조정
    # https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html
    frame = cv2.resize(frame, None, fx=scaling_factor, 
            fy=scaling_factor, interpolation=cv2.INTER_AREA) 
    ## fx - horizontal, fy = vertical, 
    ## interpolation(보간법) : INTER_AREA - resampling using pixel area relation
    

    # 흑백으로 전환
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)

    return gray 

if __name__=='__main__':
    # 비디오 캡쳐 오브젝트 초기화
    cap = cv2.VideoCapture(0)

    # 이미지 비율 정의 ( 출력 화면 크기가 변한다. 1.5쯤 하면 맥 13인치 전체 크기)
    scaling_factor = 1.0
    
    ## 첫번째 프레임 변수 초기화
    # Grab the current frame (리스트 형태로 프레임을 저장하게 된다.)
    prev_frame = get_frame(cap, scaling_factor) 
    ##print(prev_frame)
    # Grab the next frame
    cur_frame = get_frame(cap, scaling_factor) 
    # Grab the frame after that
    next_frame = get_frame(cap, scaling_factor) 

    # Keep reading the frames from the webcam 
    # until the user hits the 'Esc' key
    while True:
        # Display the frame difference
        cv2.imshow('Object Movement', frame_diff(prev_frame, 
                cur_frame, next_frame))

        # Update the variables
        prev_frame = cur_frame
        cur_frame = next_frame 

        # Grab the next frame
        next_frame = get_frame(cap, scaling_factor)

        # Check if the user hit the 'Esc' key
        key = cv2.waitKey(10)
        if key == 27:
            break

    # Close all the windows
    cv2.destroyAllWindows()
    
    


### 2. 색 공간을 이용한 물체 추적 기법
https://www.youtube.com/watch?v=hWcxEkWLm9o
https://www.youtube.com/watch?v=9qky6g8NRmI


- RGB > HSV로 변환
- 추적할 물체의 색상에 대한 임계값(threshold)를 이용하여 물체를 추적

- <b>함수</b> 그리고 <b>functions</b>    
    - cv2.inRange
    - median_blurs : Blurs an image using the median filter.
    - cv2.imshow : show images
    - cv2.waitkey : delay – Delay in milliseconds. 0 is the special value that means “forever”.
    
https://docs.opencv.org/2.4/modules/highgui/doc/user_interface.html

In [20]:
import cv2
import numpy as np

def get_frame(cap, scaling_factor):
    # 현재 비디오에서 프레임 읽기
    _, frame = cap.read()

    # 이미지 크기 조정
    # https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html
    frame = cv2.resize(frame, None, fx=scaling_factor, 
            fy=scaling_factor, interpolation=cv2.INTER_AREA)

    return frame

if __name__=='__main__':
    # 초기화
    cap = cv2.VideoCapture(0)

    # 출력창 크기 설정
    scaling_factor = 0.5

    # Keep reading the frames from the webcam 
    # until the user hits the 'Esc' key
    while True:
        # 프레임 가져오기
        frame = get_frame(cap, scaling_factor) 

        # hsv 컬러로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # 사람 피부색에 가까운 HSV 값을 설정
        lower = np.array([0, 70, 60])
        upper = np.array([50, 150, 255])

        # hsv 컬러상에서 lower~upper사이의 element들을 선별
        mask = cv2.inRange(hsv, lower, upper)

        # bit 연산을 통해 원본에서 선별된 element들을 출력
        img_bitwise_and = cv2.bitwise_and(frame, frame, mask=mask)

        # Run median blurring
        # Blurs an image using the median filter.
        # http://terms.naver.com/entry.nhn?docId=2762767&cid=50307&categoryId=50307
        img_median_blurred = cv2.medianBlur(img_bitwise_and, 5)

        # Display the input and output
        cv2.imshow('Input', frame)
        cv2.imshow('Output', img_median_blurred)

        # Check if the user hit the 'Esc' key
        c = cv2.waitKey(5) 
        if c == 27:
            break

    # Close all the windows
    cv2.destroyAllWindows()


### 3. 배경분리법
https://www.youtube.com/watch?v=KRKKektCcok

https://docs.opencv.org/3.3.0/db/d5c/tutorial_py_bg_subtraction.html

- 주어진 영상에서 배경에 해당하는 모델을 만들고, 이를 이용해 움직이는 물체를 추출하는 기법
- 정적인 배경에서 움직이는 물체를 감지하는 데 뛰어난 성능을 발휘
- 배경모델을 만든 후 이 모델이 표현하는 배경을 현재 프레임에서 제거해 전경(foreground)만을 추출
- 배경모델을 실시간으로 업데이트 한다는 점에서 프레임 차이법과의 차이점 > 기준점이 계속 변하는 적응형 알고리즘으로 구현

- 함수 & 객체
    - 배경제거 관련 객체 : createBackgroundSubtractorMOG2
    - bg_subtractor.apply( frame, learning_rate )

In [22]:
import cv2
import numpy as np

# Define a function to get the current frame from the webcam
def get_frame(cap, scaling_factor):
    # Read the current frame from the video capture object
    _, frame = cap.read()

    # Resize the image
    frame = cv2.resize(frame, None, fx=scaling_factor, 
            fy=scaling_factor, interpolation=cv2.INTER_AREA)

    return frame

if __name__=='__main__':
    # Define the video capture object
    cap = cv2.VideoCapture(0)

    # Define the background subtractor object
    bg_subtractor = cv2.createBackgroundSubtractorMOG2()
  
    ## 학습에 사용할 이전 프레임의 갯수
    ## 이를 통해 알고리즘 학습속도 제어
    ## 히스토리값이 클수록 학습 속도가 느려짐 
    history = 100

    # Define the learning rate
    learning_rate = 1.0/history

    # Keep reading the frames from the webcam 
    # until the user hits the 'Esc' key
    while True:
        # Grab the current frame
        frame = get_frame(cap, 0.5)

        # 
        mask = bg_subtractor.apply(frame, learningRate=learning_rate)

        # Convert grayscale image to RGB color image
        mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)

        # Display the images
        cv2.imshow('Input', frame)
        cv2.imshow('Output', mask & frame) ## &을 통해 비트 연산 가능

        # Check if the user hit the 'Esc' key
        c = cv2.waitKey(10)
        if c == 27:
            break

    # Release the video capture object
    cap.release()
    
    # Close all the windows
    cv2.destroyAllWindows()


### 4. 캠시프트 알고리즘을 통한 인터렉티브 방식 물체 추적기
https://www.youtube.com/watch?v=iBOlbs8i7Og

- 색 공간 기반 추적기법은 색상을 임의로 지정해주어야 한다는 한계가 존재
- mean shift를 개선한 버전의 알고리즘인 캠시프트
- 관심영역을 설정 : 추적할 물체 주변에 경계선을 그은 것
    - 해당 영역에 대한 컬러 히스토 그램을 기준으로 점의 집합을 선택 및 점들의 중심 구하기
    - 점들의 중심이 관심영역의 기하 중심에있다면 물체는 이동하지 않았다고 판단
    - 점들의 중심점이 관심영역의 중심과 다르다면 물체가 이동했다고 판단 > 경계선을 이동
    
- 함수
    - cv2.VideoCapture(0)
    - read()
    - cv2.namedWindow('Object Tracker')
    - cv2.setMouseCallback('Object Tracker', self.mouse_event)
    -


https://docs.opencv.org/3.4.0/db/df8/tutorial_py_meanshift.html

In [38]:
import cv2
import numpy as np

# Define a class to handle object tracking related functionality
class ObjectTracker(object):
    def __init__(self, scaling_factor=0.5):
        # 초기화
        self.cap = cv2.VideoCapture(0)

        # 현재 프레임 캡쳐
        _, self.frame = self.cap.read()

        # 출력창 크기 조절 인자
        self.scaling_factor = scaling_factor

        # Resize the frame
        self.frame = cv2.resize(self.frame, None, 
                fx=self.scaling_factor, fy=self.scaling_factor, 
                interpolation=cv2.INTER_AREA)

        # 프레임 표시할 창 띄우기
        cv2.namedWindow('Object Tracker')

        # 마우스로 입력받기 위한 마우스 콜백함수 설정
        cv2.setMouseCallback('Object Tracker', self.mouse_event)

        # 직사각형 모양의 선택 영역에 대한 변수 초기화
        self.selection = None
        # 시작 위치에 대한 변수 초기화
        self.drag_start = None
        # 추적 상태에 대한 변수 초기화
        self.tracking_state = 0

    # 마우스 이벤트 추적용 메소드
    def mouse_event(self, event, x, y, flags, param):
        ## x,y 좌표를 16비트 numpy 정수로 변환
        x, y = np.int16([x, y]) 
        ##print(x, y)

        ## 마우스 클릭 발생시 실행
        if event == cv2.EVENT_LBUTTONDOWN:
            self.drag_start = (x, y)
            self.tracking_state = 0

        ## 드래그 실행시 & 영역 설정 종료시 
        if self.drag_start:
            if flags & cv2.EVENT_FLAG_LBUTTON:
                # Extract the dimensions of the frame
                h, w = self.frame.shape[:2]

                # Get the initial position
                xi, yi = self.drag_start

                # Get the max and min values
                x0, y0 = np.maximum(0, np.minimum([xi, yi], [x, y]))
                x1, y1 = np.minimum([w, h], np.maximum([xi, yi], [x, y]))

                # Reset the selection variable
                self.selection = None

                # Finalize the rectangular selection
                if x1-x0 > 0 and y1-y0 > 0:
                    self.selection = (x0, y0, x1, y1)

            else:
                # If the selection is done, start tracking  
                self.drag_start = None
                if self.selection is not None:
                    self.tracking_state = 1

    # Method to start tracking the object
    def start_tracking(self):
        # Iterate until the user presses the Esc key
        while True:
            # Capture the frame from webcam
            _, self.frame = self.cap.read()
            
            # Resize the input frame
            self.frame = cv2.resize(self.frame, None, 
                    fx=self.scaling_factor, fy=self.scaling_factor, 
                    interpolation=cv2.INTER_AREA)

            # Create a copy of the frame
            vis = self.frame.copy()

            # Convert the frame to HSV colorspace
            hsv = cv2.cvtColor(self.frame, cv2.COLOR_BGR2HSV)

            # Create the mask based on predefined thresholds
            mask = cv2.inRange(hsv, np.array((0., 60., 32.)), 
                        np.array((180., 255., 255.)))

            # Check if the user has selected the region
            if self.selection:
                # Extract the coordinates of the selected rectangle
                x0, y0, x1, y1 = self.selection

                # Extract the tracking window
                self.track_window = (x0, y0, x1-x0, y1-y0)

                # Extract the regions of interest 
                hsv_roi = hsv[y0:y1, x0:x1]
                mask_roi = mask[y0:y1, x0:x1]

                # Compute the histogram of the region of 
                # interest in the HSV image using the mask
                hist = cv2.calcHist( [hsv_roi], [0], mask_roi, 
                        [16], [0, 180] )

                # Normalize and reshape the histogram
                cv2.normalize(hist, hist, 0, 255, cv2.NORM_MINMAX);
                self.hist = hist.reshape(-1)

                # Extract the region of interest from the frame
                vis_roi = vis[y0:y1, x0:x1]

                # Compute the image negative (for display only)
                cv2.bitwise_not(vis_roi, vis_roi)
                vis[mask == 0] = 0

            # Check if the system in the "tracking" mode
            if self.tracking_state == 1:
                # Reset the selection variable
                self.selection = None
                
                # Compute the histogram back projection
                hsv_backproj = cv2.calcBackProject([hsv], [0], 
                        self.hist, [0, 180], 1)

                # Compute bitwise AND between histogram 
                # backprojection and the mask
                hsv_backproj &= mask

                # Define termination criteria for the tracker
                term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                        10, 1)

                # Apply CAMShift on 'hsv_backproj'
                track_box, self.track_window = cv2.CamShift(hsv_backproj, 
                        self.track_window, term_crit)

                # Draw an ellipse around the object
                cv2.ellipse(vis, track_box, (0, 255, 0), 2)

            # Show the output live video
            cv2.imshow('Object Tracker', vis)

            # Stop if the user hits the 'Esc' key
            c = cv2.waitKey(5)
            if c == 27:
                break

        # Close all the windows
        cv2.destroyAllWindows()

if __name__ == '__main__':
	# Start the tracker
    ObjectTracker().start_tracking()



### 5. 광학 흐름 기반 추적 기법
https://www.youtube.com/watch?v=MOaKnCSejXQ

- 이미지의 특징점을 이용해서 물체를 추적하는 기법 : 현재 프레임에서 일련의 특징점을 발견하면, 이에 대한 <b>변위벡터(displacement vector)</b>를 계산해서 특징점을 추적 (변위벡터 : 움직이는 입자의 나중 위치벡터와 처음 위치벡터의 차이이다. <네이버 두산 백과사전> )
- <b>루카스-카나데(Lucas-Kanade)</b> 기법이 가장 유명 
    - 1) 현재 프레임에서 특징점을 추출
    - 2) x 3 패치(픽셀집합)을 만들기 (하나의 최소 단위)
    - 3) 프레임이 변하면서 이전 프레임의 각 픽셀집합들의 특징과 가장 유사한 픽셀집합의 특징을 다음 프레임에서 찾음
    - 4) 유사한 픽셀들 간의 이동을 추적함 (변위 벡터를 계산)


- 함수
    - cv2.calcOpticalFlowPyrLK : Lucas-kanade 방법을 이용하여 optical flow를 계산 (핵심 함수)

In [11]:
import cv2
import numpy as np

# Define a function to track the object
def start_tracking():
    # Initialize the video capture object
    cap = cv2.VideoCapture(0)

    # Define the scaling factor for the frames
    scaling_factor = 0.5

    # Number of frames to track
    num_frames_to_track = 5

    # Skipping factor
    num_frames_jump = 2

    # Initialize variables
    tracking_paths = []
    frame_index = 0

    # Define tracking parameters
    tracking_params = dict(winSize  = (11, 11), maxLevel = 2,
            criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                10, 0.03))

    # Iterate until the user hits the 'Esc' key
    while True:
        # Capture the current frame
        _, frame = cap.read()

        # Resize the frame
        frame = cv2.resize(frame, None, fx=scaling_factor, 
                fy=scaling_factor, interpolation=cv2.INTER_AREA)

        # Convert to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Create a copy of the frame
        output_img = frame.copy()

        if len(tracking_paths) > 0:
            # Get images
            prev_img, current_img = prev_gray, frame_gray

            # Organize the feature points
            feature_points_0 = np.float32([tp[-1] for tp in \
                    tracking_paths]).reshape(-1, 1, 2)

            # Compute optical flow
            feature_points_1, _, _ = cv2.calcOpticalFlowPyrLK(
                    prev_img, current_img, feature_points_0, 
                    None, **tracking_params)

            # Compute reverse optical flow
            feature_points_0_rev, _, _ = cv2.calcOpticalFlowPyrLK(
                    current_img, prev_img, feature_points_1, 
                    None, **tracking_params)

            # Compute the difference between forward and 
            # reverse optical flow
            diff_feature_points = abs(feature_points_0 - \
                    feature_points_0_rev).reshape(-1, 2).max(-1)

            # Extract the good points
            good_points = diff_feature_points < 1

            # Initialize variable
            new_tracking_paths = []

            # Iterate through all the good feature points 
            for tp, (x, y), good_points_flag in zip(tracking_paths, 
                        feature_points_1.reshape(-1, 2), good_points):
                # If the flag is not true, then continue
                if not good_points_flag:
                    continue

                # Append the X and Y coordinates and check if
                # its length greater than the threshold
                tp.append((x, y))
                if len(tp) > num_frames_to_track:
                    del tp[0]

                new_tracking_paths.append(tp)

                # Draw a circle around the feature points
                cv2.circle(output_img, (x, y), 3, (0, 255, 0), -1)

            # Update the tracking paths
            tracking_paths = new_tracking_paths

            # Draw lines
            cv2.polylines(output_img, [np.int32(tp) for tp in \
                    tracking_paths], False, (0, 150, 0))

        # Go into this 'if' condition after skipping the 
        # right number of frames
        if not frame_index % num_frames_jump:
            # Create a mask and draw the circles
            mask = np.zeros_like(frame_gray)
            mask[:] = 255
            for x, y in [np.int32(tp[-1]) for tp in tracking_paths]:
                cv2.circle(mask, (x, y), 6, 0, -1)

            # Compute good features to track
            feature_points = cv2.goodFeaturesToTrack(frame_gray, 
                    mask = mask, maxCorners = 500, qualityLevel = 0.3, 
                    minDistance = 7, blockSize = 7) 

            # Check if feature points exist. If so, append them
            # to the tracking paths
            if feature_points is not None:
                for x, y in np.float32(feature_points).reshape(-1, 2):
                    tracking_paths.append([(x, y)])

        # Update variables
        frame_index += 1
        prev_gray = frame_gray

        # Display output
        cv2.imshow('Optical Flow', output_img)

        # Check if the user hit the 'Esc' key
        c = cv2.waitKey(1)
        if c == 27:
            break

if __name__ == '__main__':
	# Start the tracker
    start_tracking()

    # Close all the windows
    cv2.destroyAllWindows()



### < face detection >
- os.chdir() 안에 haar_cascade_files가 있는 루트로 변경하고 실행하세요!

In [35]:
import cv2
import numpy as np
import os

## change directory of root path
os.chdir("/Users/EunsungJo/Documents/G/datapub/ch_13_detection/haar_cascade_files/")
os.getcwd()

# Load the Haar cascade file
face_cascade = cv2.CascadeClassifier(
        'haarcascade_frontalface_default.xml')

# Check if the cascade file has been loaded correctly
if face_cascade.empty():
	raise IOError('Unable to load the face cascade classifier xml file')

# Initialize the video capture object
cap = cv2.VideoCapture(0)

# Define the scaling factor
scaling_factor = 0.5

# Iterate until the user hits the 'Esc' key
while True:
    # Capture the current frame
    _, frame = cap.read()

    # Resize the frame
    frame = cv2.resize(frame, None, 
            fx=scaling_factor, fy=scaling_factor, 
            interpolation=cv2.INTER_AREA)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Run the face detector on the grayscale image
    face_rects = face_cascade.detectMultiScale(gray, 1.3, 5)

    # Draw a rectangle around the face
    for (x,y,w,h) in face_rects:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 3)

    # Display the output
    cv2.imshow('Face Detector', frame)

    # Check if the user hit the 'Esc' key
    c = cv2.waitKey(1)
    if c == 27:
        break

# Release the video capture object
cap.release()

# Close all the windows
cv2.destroyAllWindows()


### < face detection >

In [37]:
import cv2
import numpy as np
import os

# change directory of root path
os.chdir("/Users/EunsungJo/Documents/G/datapub/ch_13_detection/haar_cascade_files/")
os.getcwd()

# Load the Haar cascade files for face and eye
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

# Check if the face cascade file has been loaded correctly
if face_cascade.empty():
	raise IOError('Unable to load the face cascade classifier xml file')

# Check if the eye cascade file has been loaded correctly
if eye_cascade.empty():
	raise IOError('Unable to load the eye cascade classifier xml file')

# Initialize the video capture object
cap = cv2.VideoCapture(0)

# Define the scaling factor
ds_factor = 0.5

# Iterate until the user hits the 'Esc' key
while True:
    # Capture the current frame
    _, frame = cap.read()

    # Resize the frame
    frame = cv2.resize(frame, None, fx=ds_factor, fy=ds_factor, interpolation=cv2.INTER_AREA)

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Run the face detector on the grayscale image
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    # For each face that's detected, run the eye detector
    for (x,y,w,h) in faces:
        # Extract the grayscale face ROI
        roi_gray = gray[y:y+h, x:x+w]

        # Extract the color face ROI
        roi_color = frame[y:y+h, x:x+w]

        # Run the eye detector on the grayscale ROI
        eyes = eye_cascade.detectMultiScale(roi_gray)

        # Draw circles around the eyes
        for (x_eye,y_eye,w_eye,h_eye) in eyes:
            center = (int(x_eye + 0.5*w_eye), int(y_eye + 0.5*h_eye))
            radius = int(0.3 * (w_eye + h_eye))
            color = (0, 255, 0)
            thickness = 3
            cv2.circle(roi_color, center, radius, color, thickness)

    # Display the output
    cv2.imshow('Eye Detector', frame)

    # Check if the user hit the 'Esc' key
    c = cv2.waitKey(1)
    if c == 27:
        break

# Release the video capture object
cap.release()

# Close all the windows
cv2.destroyAllWindows()
